<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="https://github.com/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/figures/PDSH-cover-small.png?raw=1">

*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Handling Missing Data](03.04-Missing-Values.ipynb) | [Contents](Index.ipynb) | [Combining Datasets: Concat and Append](03.06-Concat-And-Append.ipynb) >

<a href="https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/03.05-Hierarchical-Indexing.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# 階層式(Hierarchical)索引

前面主要探討儲存在Pandas中的``Series``和``DataFrame``物件中的一維和二維資料，但實務上更常見的方式是用階層索引(multi-indexing)合併多個索引層級來進行。更高維度的資料可以被簡潔的使用一維 ``Series``和二維``DataFrame``物件來表示。

本章節，探討直接建立``MultiIndex``物件，關心在這些多索引資料的索引、切片和統計數據，以及一些有用的程序，用來在資料的簡單和階層索引表示法之間的轉換工作。

In [ ]:
import pandas as pd
import numpy as np

## 一個多重索引的Series



### 不好的方式

想要追蹤2個不同年度的州資料:使用Pandas的tuple為key

In [ ]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

這種索引機制，可以直覺的對這個序列使用多重索引進行索引或切片:

In [ ]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

但是便利性就此打住，例如:如果從2010年選所有資料，可能會需要寫蠻複雜的程式:

In [ ]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### 比較好的方法: Pandas MultiIndex


In [5]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

``MultiIndex``包含了多個索引的*層級*，在此例子中，州的名稱和年份，就好像每一個資料點的標籤，用來編碼到這些階層中。

如果使用``MultiIndex``重新索引之前的序列，可以看到資料的階層表示方式:

In [6]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

前2欄為多重索引值(州、年分)，第3欄是資料。在multi-index表示法中，任何空白的項目都表示它的值和這一行的上方相同。

現在要存取第2個索引是2010的所有資料，可以簡單用Pandas的切片就可以了:

In [7]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### 多重索引當作額外的維度

``unstack()``方法可以快速轉換多重索引的``Series``成為``DataFrame``:

In [11]:
# unstack：將資料的行“旋轉”為列
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [12]:
pop_df.stack() # Dataframe轉換成Series

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [13]:
# 舉例
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df # 用Multi-index多一個低於18歲人口的資料

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [15]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack() # 用unstack建構小於18歲人口比例的Dataframe

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


可幫助我們簡便快速操作更高維度的資料

## MultiIndex建立的方法

最直接的方式為傳遞一個2個或多個索引陣列的list給建構子:

In [16]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.818134  0.172405
  2  0.277716  0.642371
b 1  0.223707  0.499840
  2  0.831770  0.435383

同樣的，如果傳遞一個正確的tuple作為key的字典，Pandas可自動辨識出來，預設用``MultiIndex``建立:

In [17]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

然而，有時候透過明確的設定以建立``MultiIndex``會很有用，以下為這些方法的其中2個例子:

### 明確指定MultiIndex建構子(constructors)

為了讓建構索引更具彈性，可以使用``pd.MultiIndex``中的類別方法建構子

In [18]:
# 舉例:在每一層賦予索引值，從一個簡單的陣列list建立Multiindex
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

也可以在每一個點中給予多重索引值，從一個tuple的list中建構:

In [23]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

### 多重索引的階層名稱

可以藉由傳遞``names``參數到任一個之前的``MultiIndex``建構子完成這項工作，或是在之後使用index的``names``屬性設定:

In [25]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 欄的MultiIndex


In [37]:
# 階層式的索引和欄。product(元素間兩兩組合)
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# 仿製一些資料
data = np.round(np.random.randn(4, 6), 1) ## np.round: Return a sample (or samples) from the “standard normal” distribution.
data[:, ::2] *= 10
data += 37

# 建立DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      28.0  38.4  41.0  36.3  34.0  37.5
     2      33.0  37.1  54.0  38.6  42.0  37.3
2014 1      53.0  35.7  19.0  35.9  32.0  36.6
     2      43.0  38.1  30.0  38.2  37.0  36.5

使用這個方式，就可以使用姓名索引最上層的欄，即可得到存入此人資訊的一個完整的``DataFrame``:

In [ ]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      32.0  36.7
     2      50.0  35.0
2014 1      39.0  37.8
     2      48.0  37.3

對於包含多重標籤的資料，使用階層式會非常方便!

## MultiIndex的索引和切片


### 多重索引的Series

In [38]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

可藉由多個名詞的索引，存取單一元素:

In [39]:
pop['California', 2000]

33871648

也可以部分索引(partial indexing)，結果會是較低層級的Series

In [40]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

也可使用部分切片(partial slicing)

In [41]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

對於已經排序的索引，可藉由傳遞一個空的切片放在第一個索引，在比較低層級的地方執行部分索引:

In [42]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [43]:
# 使用布林遮罩
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [44]:
# Fancy索引
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### 多重索引的DataFrames



In [45]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      28.0  38.4  41.0  36.3  34.0  37.5
     2      33.0  37.1  54.0  38.6  42.0  37.3
2014 1      53.0  35.7  19.0  35.9  32.0  36.6
     2      43.0  38.1  30.0  38.2  37.0  36.5

In [46]:
health_data['Guido', 'HR']

year  visit
2013  1        41.0
      2        54.0
2014  1        19.0
      2        30.0
Name: (Guido, HR), dtype: float64

In [47]:
# 單一索引
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      28.0  38.4
     2      33.0  37.1

這些indexer提供基於二維資料的一個陣列型式的檢視，但是在 ``loc``或 ``iloc``中的每一個個別的索引，都可以傳入一個多重索引的tuple:

In [48]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        28.0
      2        33.0
2014  1        53.0
      2        43.0
Name: (Bob, HR), dtype: float64

在這些索引tuple中進行切片並不是很方便，在一個tuple中建立一個切片會導致語法錯誤:

In [49]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: ignored

In [50]:
# Pandas的IndexSlice以建立想要的切片
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,28.0,41.0,34.0
2014,1,53.0,19.0,32.0


## 重排列多重索引


### 已排序和未排序的索引

如果索引是未排序的，許多Multiindex的切片操作都會失敗。

In [52]:
# 舉例:未依詞典順序排列
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index) # random.rand():Create an array of the given shape and populate it with random samples from a uniform distribution over [0, 1).
data.index.names = ['char', 'int']
data

char  int
a     1      0.916793
      2      0.049814
c     1      0.445409
      2      0.575228
b     1      0.774990
      2      0.206177
dtype: float64

In [53]:
# 試圖去取得索引的部分切片，就會產生錯誤
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Pandas提供許多方便的程序可以排序，例如:``DataFrame``的``sort_index()``和``sortlevel()``

In [54]:
data = data.sort_index()
data

char  int
a     1      0.916793
      2      0.049814
b     1      0.774990
      2      0.206177
c     1      0.445409
      2      0.575228
dtype: float64

In [55]:
data['a':'b']

char  int
a     1      0.916793
      2      0.049814
b     1      0.774990
      2      0.206177
dtype: float64

### 索引的Stacking和unstacking


In [56]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [57]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [58]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 索引的Setting和resetting

``reset_index``:把標籤轉換成欄。
``set_index``:把欄轉換成標籤

In [59]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [60]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## 在多重索引上的資料聚合處理

之前Pandas有一些內建的資料聚合方法，像是 ``mean()``、``sum()``、以及 ``max()``。

對階層式索引的資料，這些函數可以傳入level參數，去控制哪一個資料子集合是要被計算的。


In [61]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      28.0  38.4  41.0  36.3  34.0  37.5
     2      33.0  37.1  54.0  38.6  42.0  37.3
2014 1      53.0  35.7  19.0  35.9  32.0  36.6
     2      43.0  38.1  30.0  38.2  37.0  36.5

In [62]:
# 透過索引層名稱來計算每一年2次測量的平均值
data_mean = health_data.mean(level='year')
data_mean

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  


subject   Bob        Guido          Sue       
type       HR   Temp    HR   Temp    HR   Temp
year                                          
2013     30.5  37.75  47.5  37.45  38.0  37.40
2014     48.0  36.90  24.5  37.05  34.5  36.55

In [63]:
# 以欄計算平均值
data_mean.mean(axis=1, level='type')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  


type,HR,Temp
year,,
2013,38.666667,37.533333
2014,35.666667,36.833333
